# impacts of STAC item footprint size on dynamic tiling query performance

**TL;DR:** If you have any control over the geographic footprint of the assets that you are cataloging with `pgstac` and you want to serve visualizations with a dynamic tiling application, try to maximize the size of the assets!

Dynamic tiling applications like [`titiler-pgstac`](https://github.com/stac-utils/titiler-pgstac) send many queries to a `pgstac` database and clients are very sensitive to performance so it is worth considering a few basic ideas when building collections and items that may be used in this way.

`pgstac`'s query functions perform relatively expensive spatial intersection operations so the fewer items there are in a collection x datetime partition, the faster the query will be. This is not a `pgstac`-specific problem (any application that needs to perform spatial intersections will take longer as the number of calculations increases), but it is worth demonstrating the influence of these factors in the dynamic tiling context.

In [ ]:
import json
import math
import uuid
from datetime import datetime, timezone
from typing import Any, Dict, Generator, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
from folium import Map, GeoJson, LayerControl
from matplotlib.colors import LogNorm


XMIN, YMIN = 0, 0
AOI_WIDTH = 50
AOI_HEIGHT = 50
ITEM_WIDTHS = [0.5, 1, 2, 4, 6, 8, 10]

def generate_items(
    item_size: Tuple[float, float],
    collection_id: str,
) -> Generator[Dict[str, Any], None, None]:
    item_width, item_height = item_size

    cols = math.ceil(AOI_WIDTH / item_width)
    rows = math.ceil(AOI_HEIGHT / item_height)

    # Generate items for each grid cell
    for row in range(rows):
        for col in range(cols):
            left = XMIN + (col * item_width)
            bottom = YMIN + (row * item_height)
            right = left + item_width
            top = bottom + item_height

            yield {
                "type": "Feature",
                "stac_version": "1.0.0",
                "id": str(uuid.uuid4()),
                "collection": collection_id,
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [left, bottom],
                            [right, bottom],
                            [right, top],
                            [left, top],
                            [left, bottom],
                        ],
                    ],
                },
                "bbox": [left, bottom, right, top],
                "properties": {
                    "datetime": datetime.now(timezone.utc).isoformat(),
                },
            }



def load_benchmark_results() -> pd.DataFrame:
    """Load benchmark results from JSON file into a pandas DataFrame."""
    with open("./benchmark.json") as f:
        data = json.load(f)

    # Extract the benchmarks into a list of records
    records = []
    for benchmark in data["benchmarks"]:
        record = {
            "item_width": benchmark["params"]["item_width"],
            "zoom": benchmark["params"]["zoom"],
            "mean": benchmark["stats"]["mean"],
            "stddev": benchmark["stats"]["stddev"],
            "median": benchmark["stats"]["median"],
        }

        records.append(record)

    return pd.DataFrame(records).sort_values(by=["item_width", "zoom"])


stac_items = {
    item_width: list(
        generate_items(
            (item_width, item_width),
            f"{item_width} degrees"
        )
    )
    for item_width in ITEM_WIDTHS
}

df = load_benchmark_results()

## Scenario
Imagine you have a continental-scale dataset of gridded data that will be stored as cloud-optimized geotiffs (COGs) and you get to decide how the individual files will be spatially arranged and cataloged in a `pgstac` database. You could make items as small as 0.5 degree squares or as large as 10 degree squares. In this case the assets will be non-overlapping rectangular grids.

The assets will be publicly accessible, so smaller file sizes might be useful for some applications/users, but since the data will be stored as COGs and we also want to be able to serve raster tile visualizations in a web map with `titiler-pgstac`, smaller file sizes are not very important. However, the processing pipleline that generates the assets might have some resource constraints that push you to choose a smaller tile size.

Consider the following options for tile sizes:

In [ ]:
pd.DataFrame(
    {"tile width (degrees)": item_width, "# items": len(items)}
    for item_width, items in stac_items.items()
)

The number of items is inversely proportional to the square of the tile width which means that small changes in tile size can have a large impact on the eventual number of items in your catalog!

This map shows the spatial arrangement of the items for a range of tile sizes:

In [ ]:
m = Map([25, 25], zoom_start=3)
for item_width in ITEM_WIDTHS:
    layer_name = f"{item_width} degrees"
    geojson = GeoJson(
        {
            "type": "FeatureCollection",
            "features": stac_items[item_width],
        },
        name=layer_name,
        overlay=True,
        show=False,
    )
    geojson.add_to(m)
    
LayerControl(collapsed=False, position="topright").add_to(m)

m

## Performance comparison
To simulate the performance of queries made by a dynamic tiling application we have prepared a benchmarking procedure that uses the `pgstac` function `xyzsearch` to run an item query for an XYZ tile. By iterating over many combinations of tile sizes and zoom levels we can examine the response time with respect to item footprint size and tile zoom level. 

This figure shows average response time for `xyzsearch` to return a complete set of results for each zoom level for the range of item tile widths:

In [ ]:
ax = sns.heatmap(
    df.pivot(index="item_width", columns="zoom", values="median"),
    norm=LogNorm(vmin=1e-2, vmax=1e1),
    cbar_kws={
        "ticks": np.logspace(-2, 0, num=3),
        "format": "%.1e",
    }
)
ax.set(xlabel="zoom level", ylabel="item tile width")
ax.xaxis.tick_top()
ax.xaxis.set_label_position("top")
display(ax)

Without details about the resource configuration for a specific `pgstac` deployment it is hard to say which zoom level becomes inoperable for a given tile size, but queries that take >0.5 seconds in this test would probably yield poor results in a deployed context.